# Section 0: Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
%matplotlib inline

# Import API key
from config import gimgkey

# Section 1: Build Dataset with API Call

In [25]:
# Read stables.csv to get list of fighters
df = pd.read_csv('Stables.csv')
df.head()

,id,rank,fighter,dojo,district,birth_date,height,weight,lat,lng
0,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,4/3/53,186.0,133.0,43.043111,141.316019
1,4122,M14e,Kurosegawa,Isegahama,Tokyo,5/13/51,184.0,130.0,32.413809,131.649506
2,4103,M14w,Banryuyama,Mihogaseki,Hyogo,5/4/51,177.5,135.0,35.495838,136.918181
3,4050,M8e,Takamiyama,Takasago,U.S.A.,6/16/44,192.0,205.0,34.766101,134.790601
4,4077,M10w,Fujizakura,Takasago,Yamanashi,2/9/48,180.0,139.0,34.766101,134.790601


In [38]:
fighter_names = df['fighter']
fighter_names.head()

0    Wakanohana
1    Kurosegawa
2    Banryuyama
3    Takamiyama
4    Fujizakura
Name: fighter, dtype: object

In [39]:
len(fighter_names)

331

In [46]:
fighter_img = pd.DataFrame({'fighter_name':fighter_names})

In [48]:
fighter_img.head()

,fighter_name
0,Wakanohana
1,Kurosegawa
2,Banryuyama
3,Takamiyama
4,Fujizakura


In [49]:
#Perform API Call

#Create needed columns to store data for each fighter_image
fighter_img['img_url'] =''

#Perform API call to get information for each stable
base_url = 'https://app.zenserp.com/api/v2/search'

params = {"apikey": gimgkey, 'tbm':'isch','num':'1'}

#Iterate through the list of fighetrs to retrieve their information
for i, row in fighter_img.iterrows():
    
    params['q'] = row['fighter_name']

    response = requests.get(base_url,params=params).json()
    
    try:
        fighter_img.loc[i,'img_url']= response['image_results'][0]['sourceUrl']
        
    except:
        #Assign NaN to Fighters not found
        fighter_img.loc[i,'img_url']= np.nan
    



In [52]:
fighter_img.head()

,fighter_name,img_url
0,Wakanohana,https://upload.wikimedia.org/wikipedia/commons...
1,Kurosegawa,https://upload.wikimedia.org/wikipedia/commons...
2,Banryuyama,http://www.sumoforum.net/tegata/imgdir/banryuy...
3,Takamiyama,https://images-na.ssl-images-amazon.com/images...
4,Fujizakura,http://cdn.shopify.com/s/files/1/0098/8391/020...


In [53]:
fighter_img.to_csv('fighter_img.csv',index=False)

# Section 2: Write to Postgres DB

In [6]:
fighter_img = pd.read_csv('fighter_img.csv')

In [7]:
fighter_img.head()

,fighter_name,image_url
0,Wakanohana,https://upload.wikimedia.org/wikipedia/commons...
1,Kurosegawa,https://upload.wikimedia.org/wikipedia/commons...
2,Banryuyama,http://www.sumoforum.net/tegata/imgdir/banryuy...
3,Takamiyama,https://images-na.ssl-images-amazon.com/images...
4,Fujizakura,http://cdn.shopify.com/s/files/1/0098/8391/020...


In [8]:
#Connect to Postgress
rds_connection_string = "carlospazos@127.0.0.1:5432/sumo_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [9]:
engine.table_names()

['tournament_results', 'stables', 'image']

In [10]:
#Insert dataframes into Postgress DB
fighter_img.to_sql(name='image', con=engine, if_exists='append', index=False)